# Introducing Databases and SQL

teaching: 60
exercises: 5

## Questions

- "What is a relational database and why should I use it?"
- "What is SQL?"

## Objectives

- "Describe why relational databases are useful."
- "Create and populate a database from a text file."
- "Define SQLite data types."

## Key points

- "SQL allows us to select and group subsets of data, do math and other calculations, and combine data."
- "A relational database is made up of tables which are related to each other by shared keys."
- "Different database management systems (DBMS) use slightly different vocabulary, but they are all based on the same ideas."

# Motivation

To start, let's orient ourselves in our project workflow.  Previously,
we used Excel and OpenRefine to go from messy, human created data
to cleaned, computer-readable data.  Now we're going to move to the next piece
of the data workflow, using the computer to read in our data, and then
use it for analysis and visualization.

## What is SQL?

SQL stands for Structured Query Language. SQL allows us to interact with relational databases through queries.
These queries can allow you to perform a number of actions such as: insert, select, update and delete information in a database.


## Dataset Description

The data we will be using is a time-series for a small mammal community in
southern Arizona. This is part of a project studying the effects of rodents and
ants on the plant community that has been running for almost 40 years.  The
rodents are sampled on a series of 24 plots, with different experimental
manipulations controlling which rodents are allowed to access which plots.

This is a real dataset that has been used in over 100 publications. We've
simplified it for the workshop, but you can download the
[full dataset](http://esapubs.org/archive/ecol/E090/118/) and work with it using
exactly the same tools we'll learn about today.

## Questions

Let's look at some of the cleaned spreadsheets you downloaded during [Setup](../setup.html) to complete this challenge. You'll need the following three files:

* `surveys.csv`
* `species.csv`
* `plots.csv`

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///../data/data.db

In [ ]:
%%sql
DROP TABLE IF EXISTS surveys;
CREATE TABLE surveys( 
    record_id INTEGER, 
    month INTEGER, 
    day INTEGER, 
    year INTEGER, 
    plot_id INTEGER,
    species_id TEXT, 
    sex TEXT, 
    hindfoot_length REAL, 
    weight REAL );

> ## Challenge
>
> - Create the `plots` and `species` tables

In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_master
WHERE
    type='table'

In [ ]:
%%sql
PRAGMA table_info(surveys);

In [ ]:
%%sql
DELETE FROM species

In [ ]:
! echo "SELECT COUNT(*) FROM species" | sqlite3 -csv -cmd ".import --skip 1 ../data/species.csv species" ../data/data.db

In [ ]:
%%sql
SELECT
    *
FROM
    species
LIMIT 10